In [ ]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

"""
Load Vehicle recall

"""

# Import libraries

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import pandas as pd
import aerospike
import json
import csv

print(__doc__)

def load():
    
    global rec
    rec = {}

    csvfile  = open('/home/azureuser/Aerospike/data/customer/vehicle_recall.csv', "rb")
    reader = csv.reader(csvfile)

    config = {
                'hosts': [ ('127.0.0.1', 3000) ]
            }

    # Create a client and connect it to the cluster
    try:
        client = aerospike.client(config).connect()
    except:
        import sys
        print("failed to connect to the cluster with", config['hosts'])
        sys.exit(1)
            
    rownum = 0
    for row in reader:
        # Save First Row with headers
        if rownum == 0:
            header = row
            header = [x.lower() for x in header]
        else:
            colnum = 0
            for col in row:
                # print (rownum,header[colnum],col)
                rec[header[colnum]] = col
                colnum += 1
        rownum += 1
        # print(rownum,rec)
        if rec:
            try:
                client = aerospike.client(config).connect()
                # Write a record
                client.put(('test', 'demo', str(rownum)), rec)
            except Exception as e:
                import sys
                print("error: {0}".format(e), file=sys.stderr)
        
            # Read a record after insert for verification
            (key, metadata, record) = client.get(('test', 'demo', str(rownum)))
            print (key, record)
        
            # Close the connection to the Aerospike cluster
            client.close()
        rec = {}

def delete():
    
    global rec
    rec = {}

    csvfile  = open('/home/azureuser/Aerospike/data/customer/vehicle_recall.csv', "rb")
    reader = csv.reader(csvfile)

    config = {
                'hosts': [ ('127.0.0.1', 3000) ]
            }

    # Create a client and connect it to the cluster
    try:
        client = aerospike.client(config).connect()
    except:
        import sys
        print("failed to connect to the cluster with", config['hosts'])
        sys.exit(1)
            
    rownum = 0
    for row in reader:
        # Save First Row with headers
        if rownum == 0:
            header = row
            header = [x.lower() for x in header]
        else:
            colnum = 0
            for col in row:
                # print (rownum,header[colnum],col)
                rec[header[colnum]] = col
                colnum += 1
        rownum += 1
        # print(rownum,rec)
        if rec:
            try:
                client = aerospike.client(config).connect()
                # Write a record
                client.remove(('test', 'demo', str(rownum)))
            except Exception as e:
                import sys
                print("error: {0}".format(e), file=sys.stderr)
        
            # Close the connection to the Aerospike cluster
            client.close()
        rec = {}

def drive(mode):
    if mode == 'load':
        load()
    else:
        delete()
        
if __name__ == '__main__':
    drive('delete')